This notebook will go over one of the easiest ways to graph data from your [Amazon Redshift data warehouse](http://aws.amazon.com/redshift/) using [Plotly's public platform](https://plot.ly/) for publishing beautiful, interactive graphs from Python to the web.

[Plotly's Enterprise platform](https://plot.ly/product/enterprise/) allows for an easy way for your company to build and share graphs without the data leaving your servers.

In [1]:
from __future__ import print_function #python 3 support

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
import pandas as pd
import os
import requests
requests.packages.urllib3.disable_warnings() # this squashes insecure SSL warnings - DO NOT DO THIS ON PRODUCTION!

We'll me using [Amazon's Sample Data](http://docs.aws.amazon.com/redshift/latest/gsg/rs-gsg-create-sample-db.html) for this notebook. Although we won't be connecting through a JDBC/ODBC connection we'll be using the [fantastic psycopg2 package](http://initd.org/psycopg/docs/index.html) with the [SQLAlchemy](http://www.sqlalchemy.org/) package and pandas to make it simple to query and analyze our data.

You'll need your [Redshift Endpoint URL](http://docs.aws.amazon.com/redshift/latest/gsg/rs-gsg-connect-to-cluster.html) in order to access your redshift instance. I've obscured mine below but yours will be in a format similar to `datawarehouse.some_chars_here.region_name.redshift.amazonaws.com` .

Then connecting is as easy as entering in the informaiton on your cluster properties, you'll need the username, password, port, hose and database name. As a friendly reminder, remember to keep login information out of version control.

In [2]:
redshift_endpoint = os.getenv("REDSHIFT_ENDPOINT")
redshift_user = os.getenv("REDSHIFT_USER")
redshift_pass = os.getenv("REDSHIFT_PASS")
port = 5439
dbname = 'dev'

There are many ways to connect to a Redshift database and I've included two below. We can use the SQLAlchemy package or we can use the raw psycopg2 package. Both will allow us to execute SQL queries and get results however the SQLAlchemy engine makes it easier to directly return our data as a dataframe using pandas.

####SQLAlchemy

In [3]:
from sqlalchemy import create_engine
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user, redshift_pass, redshift_endpoint, port, dbname)
engine = create_engine(engine_string)

####Psycopg2

In [4]:
import psycopg2
conn = psycopg2.connect(
    host=redshift_endpoint, 
    user=redshift_user, 
    port=port, 
    password=redshift_pass, 
    dbname=dbname)
cur = conn.cursor() # create a cursor for executing queries

###Loading in Data

This next section goes over loading in the sample data from Amazon's sample database. If you're going to work with your own data you can easily skip this section.

-----------------START DATA LOADING-----------------

In [7]:
aws_key = os.getenv("AWS_ACCESS_KEY_ID") # needed to access S3 Sample Data
aws_secret = os.getenv("AWS_SECRET_ACCESS_KEY")

base_copy_string = """copy %s from 's3://awssampledbuswest2/tickit/%s.txt' 
credentials 'aws_access_key_id=%s;aws_secret_access_key=%s' 
delimiter '%s';""" # the base COPY string that we'll be using

#easily generate each table that we'll need to COPY data from
tables = ["users", "venue", "category", "date", "event", "listing"]
data_files = ["allusers_pipe", "venue_pipe", "category_pipe", "date2008_pipe", "allevents_pipe", "listings_pipe"]
delimiters = ["|", "|", "|", "|", "|", "|", "|"]

#the generated COPY statements we'll be using to load data;
copy_statements = []
for tab, f, delim in zip(tables, data_files, delimiters):
    copy_statements.append(base_copy_string % (tab, f, aws_key, aws_secret, delim))

# add in Sales data, delimited by '\t'
copy_statements.append("""copy sales from 's3://awssampledbuswest2/tickit/sales_tab.txt' 
credentials 'aws_access_key_id=%s;aws_secret_access_key=%s' 
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS';""" % (aws_key, aws_secret))

In [9]:
# Create Table Statements
cur.execute("""
create table users(
	userid integer not null distkey sortkey,
	username char(8),
	firstname varchar(30),
	lastname varchar(30),
	city varchar(30),
	state char(2),
	email varchar(100),
	phone char(14),
	likesports boolean,
	liketheatre boolean,
	likeconcerts boolean,
	likejazz boolean,
	likeclassical boolean,
	likeopera boolean,
	likerock boolean,
	likevegas boolean,
	likebroadway boolean,
	likemusicals boolean);

create table venue(
	venueid smallint not null distkey sortkey,
	venuename varchar(100),
	venuecity varchar(30),
	venuestate char(2),
	venueseats integer);

create table category(
	catid smallint not null distkey sortkey,
	catgroup varchar(10),
	catname varchar(10),
	catdesc varchar(50));

create table date(
	dateid smallint not null distkey sortkey,
	caldate date not null,
	day character(3) not null,
	week smallint not null,
	month character(5) not null,
	qtr character(5) not null,
	year smallint not null,
	holiday boolean default('N'));

create table event(
	eventid integer not null distkey,
	venueid smallint not null,
	catid smallint not null,
	dateid smallint not null sortkey,
	eventname varchar(200),
	starttime timestamp);

create table listing(
	listid integer not null distkey,
	sellerid integer not null,
	eventid integer not null,
	dateid smallint not null  sortkey,
	numtickets smallint not null,
	priceperticket decimal(8,2),
	totalprice decimal(8,2),
	listtime timestamp);

create table sales(
	salesid integer not null,
	listid integer not null distkey,
	sellerid integer not null,
	buyerid integer not null,
	eventid integer not null,
	dateid smallint not null sortkey,
	qtysold smallint not null,
	pricepaid decimal(8,2),
	commission decimal(8,2),
	saletime timestamp);""")

In [11]:
for copy_statement in copy_statements: # execute each COPY statement
    cur.execute(copy_statement)
conn.commit()

In [12]:
for table in tables + ["sales"]:
    cur.execute("select count(*) from %s;" % (table,))    
    print(cur.fetchone())
conn.commit() # make sure data went through and commit our statements permanently.

(49990L,)
(202L,)
(11L,)
(365L,)
(8798L,)
(192497L,)
(172456L,)


-----------------END DATA LOADING-----------------

Now that we've finished loading in our data, we can start running some queries to explore and present what our user's tastes and habits.

In [5]:
df = pd.read_sql_query("""
SELECT sum(likesports::int) as sports, sum(liketheatre::int) as theatre,  
sum(likeconcerts::int) as concerts, sum(likejazz::int) as jazz, 
sum(likeclassical::int) as classical, sum(likeopera::int) as opera,  
sum(likerock::int) as rock, sum(likevegas::int) as vegas,  
sum(likebroadway::int) as broadway, sum(likemusicals::int) as musical, 
state
FROM users 
GROUP BY state;
""", engine)

In [6]:
data = Data([
        Heatmap(
            z = df.drop('state', axis=1).values,
            x = df.drop('state', axis=1).columns,
            y=df.state,
            colorscale='Hot'
        )
    ])
py.iplot(data, filename='redshift/state and music taste heatmap')

*the above graph is interactive, click and drag to zoom, double click to return to initial layout, shift click to pan*

Heatmaps are great because we can easily get a sense for activity. We can see that sports aren't popular amoung our users and taht certain states seem to have the greatest popularity across the board. 

Let's explore a bit further and see which group has the highest standard deviation.

In [7]:
layout = Layout(title="Declared User Preference Box Plot", 
                yaxis=YAxis())

data = []
for pref in df.drop('state', axis=1).columns:
    data.append(Box(y=df[pref], name=pref))
    
py.iplot(
    Figure(data=data, layout=layout)
)

*the above graph is interactive, click and drag to zoom, double click to return to initial layout, shift click to pan*

Now that we've explored a little bit about the users, a logical next question is, do we list less sports events on our site? Or do we sell approximately the same amount of all event types and our users just seem to dislike sports events?

In [8]:
df = pd.read_sql_query("""
SELECT sum(event.catid) as category_sum, catname as category_name
FROM event, category
where event.catid = category.catid
GROUP BY category.catname
""", engine)

In [9]:
layout = Layout(title="Event Categories Sum", yaxis=YAxis(title="Sum"))
data = [Bar(x=df.category_name, y=df.category_sum)]
py.iplot(
    Figure(data=data, layout=layout)
)

We've got to rush to management because we've got a significant discrepancy between the types of events that we are listing and our user's preferences! Let's dive a bit deeper into the events that we're listing and when we're listing them.

In [10]:
df = pd.read_sql_query("""
SELECT sum(sales.qtysold) as quantity_sold, date.caldate  
FROM sales, date
WHERE sales.dateid = date.dateid 
GROUP BY date.caldate 
ORDER BY date.caldate asc;
""", engine)

In [11]:
layout = Layout(title="Event Sales Per Day", yaxis=YAxis(title="Sales Quantity"))
data = [Scatter(x=df.caldate, y=df.quantity_sold)]
py.iplot(Figure(data=data, layout=layout))

In [12]:
df = pd.read_sql_query("""
SELECT sum(sales.qtysold) as quantity_sold, date.caldate, category.catname as category_name  
FROM sales, date, event, category
WHERE sales.dateid = date.dateid 
AND sales.eventid = event.eventid
AND event.catid = category.catid
GROUP BY date.caldate, category_name
ORDER BY date.caldate asc;
""", engine)

In [13]:
data = []
for count, (name, g) in enumerate(df.groupby("category_name")):
    data.append(Scatter(
            name=name,
            x=g.caldate,
            y=g.quantity_sold,
            xaxis='x' + str(count + 1),
            yaxis='y' + str(count + 1)
        ))

fig = tls.get_subplots(rows=2,columns=2)
fig['layout'].update(title="Event Sales Per Day By Category")
fig['data'] += data
py.iplot(fig)

/Users/bill_chambers/.virtualenvs/plotly-notebook/lib/python2.7/site-packages/plotly/tools.py:438: UserWarning:

tools.get_subplots is depreciated. Please use tools.make_subplots instead.



In [14]:
data = []
for name, g in df.groupby("category_name"):
    data.append(Scatter(
            name=name,
            x=g.caldate,
            y=g.quantity_sold
        ))

fig = Figure()
fig['layout'].update(title="Event Sales Per Day By Category")
fig['data'] += data
py.iplot(fig)